In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.metrics import roc_curve
from sklearn import metrics

In [3]:
df = pd.read_csv('praram9_selected_1000_w_org_ratio.csv')
Thresholdfile2 = pd.read_csv('threshold_ab_10502_cardio90_correct.csv')
Thresholdfile1 = pd.read_csv('threshold_ab10502_tb105e.csv')
Thresholdfile3 = pd.read_csv('threshold_tuberculosis.csv')
GT = pd.read_csv('nipple_marker_raw.csv')

In [4]:
GT.columns

Index(['Image Index', 'Atelectasis', 'Atelectasis_Minimal', 'Cardiomegaly',
       'Cardiomegaly_Minimal', 'Consolidation', 'Consolidation_Minimal',
       'Edema', 'Edema_Minimal', 'Exclude', 'Fibrosis', 'Fibrosis_Minimal',
       'Infiltration', 'Infiltration_Minimal', 'Lung Opacity',
       'Lung Opacity_Minimal', 'Mass', 'Mass_Minimal', 'Nipple Marker',
       'Nodule', 'Nodule_Minimal', 'Pleural Effusion',
       'Pleural Effusion_Minimal', 'Tuberculosis', 'Tuberculosis_Minimal'],
      dtype='object')

In [5]:
Thresholdfile = pd.merge(Thresholdfile1,Thresholdfile2,how = 'outer', on = 'Disease')
Thresholdfile = pd.merge(Thresholdfile,Thresholdfile3,how = 'outer', on = 'Disease')
Thresholdfile.dropna(axis = 1,inplace = True)
Thresholdfile.rename({'Prob Threshold_x':'Prob Threshold','Confidence Threshold_x':'Confidence Threshold'},
                     axis = 'columns', inplace = True)

Thresholdfile['Disease'].replace('Inspectra Lung Opacity v1','Lung Opacity',inplace= True)
Thresholdfile.set_index('Disease',inplace = True)

In [6]:
GT['Lung Opacity'] = GT['Lung Opacity']+GT['Consolidation']+GT['Infiltration']
GT['Lung Opacity_Minimal'] = GT['Lung Opacity_Minimal']+GT['Consolidation_Minimal']+GT['Infiltration_Minimal']
L = []
Lm = []
for i in GT['Lung Opacity'] :
    if i == 0.0 :
        L.append(0.0)
    else :
        L.append(1.0)
for index,row in GT[['Lung Opacity_Minimal']].iterrows():
    if row[0]!= 0:
        if row[0] == GT.iloc[index]['Lung Opacity'] :
            Lm.append(1.0)
        else :
            Lm.append(0.0)
    else :
        Lm.append(0.0)
GT['Lung Opacity_Minimal'] = Lm
GT['Lung Opacity'] = L 

In [7]:
df.iloc[999]['Image Index']

'1.2.392.200046.100.14.158304376441014525168384165406783713196.png'

In [8]:
L = [] 
for i in df['Image Index']: 
    i = i[:-4]
    L.append(i)
df['Image Index'] = L 

In [9]:
GT = GT[['Image Index','Atelectasis_Minimal','Cardiomegaly_Minimal','Edema_Minimal','Lung Opacity_Minimal',
         'Mass_Minimal','Nodule_Minimal','Pleural Effusion_Minimal','Tuberculosis_Minimal',
         'Atelectasis','Cardiomegaly','Edema','Lung Opacity',
         'Mass','Nodule','Pleural Effusion','Tuberculosis','Nipple Marker']]
df = df.drop(['Atelectasis','Cardiomegaly','Edema','Lung Opacity',
              'Mass','Nodule','Pleural Effusion','Tuberculosis'],axis = 1)

In [30]:
df = pd.merge(df,GT,how = 'outer', on = 'Image Index')
df.head()

,Image Index,Accession Number,Acquisition Date,Month,PID,Inspectra Lung Opacity v1 Balanced Score,Mass Balanced Score,Nodule Balanced Score,Edema Balanced Score,Atelectasis Balanced Score,...,Tuberculosis_Minimal_y,Atelectasis_y,Cardiomegaly_y,Edema_y,Lung Opacity_y,Mass_y,Nodule_y,Pleural Effusion_y,Tuberculosis_y,Nipple Marker_y
0,1.2.392.200046.100.14.105929304388117274968456...,XR6418198-XR0046,20210716,7,XR6418198,0.868155,0.165346,0.636160,0.193228,0.391986,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,1.2.392.200046.100.14.109045240710600353114852...,XR6437573-XR0046,20211215,12,XR6437573,0.577891,0.044495,0.283493,0.077815,0.058540,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.2.392.200046.100.14.109828079333066932603687...,XR6426459-XR0373,20210911,9,XR6426459,0.073780,0.067282,0.161142,0.004501,0.027519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.2.392.200046.100.14.123036775600773791681570...,XR6432932-XR0373,20211107,11,XR6432932,0.454977,0.185435,0.139441,0.145562,0.145533,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.2.392.200046.100.14.128498105353347301482567...,B6420420-XR0046B,20211119,11,B6420420,0.089672,0.065298,0.056001,0.012179,0.029267,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df = df.sort_values(by = 'Image Index')

In [12]:
df.iloc[998]['Image Index']

'2.16.840.1.113845.1.1.1681513874441511629512690011920000000035'

In [13]:
# page 6 
# 1.2.392.200046.100.14.13199040445778684733309045085801280.png

In [14]:
L = [] 
for i in range(200) : 
               L.append('Task1370'+'Page'+ str(i))
for i in range(200,400) : 
               L.append('Task1371'+'Page'+ str(i))               
for i in range(400,600) : 
               L.append('Task1372'+'Page'+ str(i))
for i in range(600,800) : 
               L.append('Task1373'+'Page'+ str(i)) 
for i in range(800,1000) : 
               L.append('Task1374'+'Page'+ str(i))
df['Page in CVAT'] = L 

In [15]:
df.iloc[998]['Image Index']

'2.16.840.1.113845.1.1.1681513874441511629512690011920000000035'

In [16]:
dfwoNM = df[df['Nipple Marker']==0.0]
dfwNM = df[df['Nipple Marker']==1.0]

In [29]:
dfwNM[['Image Index','Page in CVAT']].head()

,Image Index,Page in CVAT
291,1.2.392.200046.100.14.105929304388117274968456...,Task1370Page0
946,1.2.392.200046.100.14.109828079333066932603687...,Task1370Page2
534,1.2.392.200046.100.14.123036775600773791681570...,Task1370Page3
313,1.2.392.200046.100.14.128620041104259305112514...,Task1370Page5
814,1.2.392.200046.100.14.131990404457786847333090...,Task1370Page6


In [18]:
def TP(df,GTcolumn):
    T = df[(df[GTcolumn+' Balanced Score']>= Thresholdfile.loc[GTcolumn]['Confidence Threshold'])&(df[GTcolumn]==1.0)]
    return T
def TN(df,GTcolumn):
    T = df[(df[GTcolumn+' Balanced Score']< Thresholdfile.loc[GTcolumn]['Confidence Threshold'])&(df[GTcolumn]==0.0)]
    return T 
def FP(df,GTcolumn):
    T = df[(df[GTcolumn+' Balanced Score']>= Thresholdfile.loc[GTcolumn]['Confidence Threshold'])&(df[GTcolumn]==0.0)]
    return T
def FN(df,GTcolumn):
    T = df[(df[GTcolumn+' Balanced Score']< Thresholdfile.loc[GTcolumn]['Confidence Threshold'])&(df[GTcolumn]==1.0)]
    return T
def CVATPage(df) : 
    return df['Page in CVAT'].values

In [19]:
CVATPage(FN(df[(df['Cardiomegaly']==1.0)&(df['Cardiomegaly_Minimal']==0.0)],'Cardiomegaly'))

array(['Task1370Page115', 'Task1370Page157', 'Task1374Page810',
       'Task1374Page893', 'Task1374Page901', 'Task1374Page934'],
      dtype=object)

In [20]:
CVATPage(FP(dfwNM,'Nodule'))

array(['Task1370Page0', 'Task1370Page9', 'Task1370Page12',
       'Task1370Page17', 'Task1370Page19', 'Task1370Page25',
       'Task1370Page26', 'Task1370Page29', 'Task1370Page30',
       'Task1370Page31', 'Task1370Page38', 'Task1370Page46',
       'Task1370Page50', 'Task1370Page56', 'Task1370Page59',
       'Task1370Page60', 'Task1370Page69', 'Task1370Page79',
       'Task1370Page93', 'Task1370Page94', 'Task1370Page105',
       'Task1370Page107', 'Task1370Page108', 'Task1370Page115',
       'Task1370Page119', 'Task1370Page129', 'Task1370Page144',
       'Task1370Page148', 'Task1370Page166', 'Task1370Page167',
       'Task1370Page180', 'Task1370Page191', 'Task1370Page196',
       'Task1370Page199', 'Task1371Page205', 'Task1371Page206',
       'Task1371Page208', 'Task1371Page212', 'Task1371Page214',
       'Task1371Page221', 'Task1371Page222', 'Task1371Page239',
       'Task1371Page243', 'Task1371Page249', 'Task1371Page259',
       'Task1373Page704', 'Task1373Page707', 'Task1373Page709'

In [21]:
CVATPage(FN(dfwoNM,'Pleural Effusion'))

array(['Task1370Page40', 'Task1370Page130', 'Task1374Page935'],
      dtype=object)

In [22]:
df[df['Page in CVAT'] == 'Task1370Page6']['Image Index'].values[0]

'1.2.392.200046.100.14.13199040445778684733309045085801280'

In [23]:
df.head()

,Image Index,Accession Number,Acquisition Date,Month,PID,Inspectra Lung Opacity v1 Balanced Score,Mass Balanced Score,Nodule Balanced Score,Edema Balanced Score,Atelectasis Balanced Score,...,Atelectasis,Cardiomegaly,Edema,Lung Opacity,Mass,Nodule,Pleural Effusion,Tuberculosis,Nipple Marker,Page in CVAT
291,1.2.392.200046.100.14.105929304388117274968456...,XR6418198-XR0046,20210716,7,XR6418198,0.868155,0.165346,0.636160,0.193228,0.391986,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,Task1370Page0
395,1.2.392.200046.100.14.109045240710600353114852...,XR6437573-XR0046,20211215,12,XR6437573,0.577891,0.044495,0.283493,0.077815,0.058540,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Task1370Page1
946,1.2.392.200046.100.14.109828079333066932603687...,XR6426459-XR0373,20210911,9,XR6426459,0.073780,0.067282,0.161142,0.004501,0.027519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Task1370Page2
534,1.2.392.200046.100.14.123036775600773791681570...,XR6432932-XR0373,20211107,11,XR6432932,0.454977,0.185435,0.139441,0.145562,0.145533,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Task1370Page3
355,1.2.392.200046.100.14.128498105353347301482567...,B6420420-XR0046B,20211119,11,B6420420,0.089672,0.065298,0.056001,0.012179,0.029267,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Task1370Page4


In [24]:
192+49

241

In [25]:
1000-241

759

In [26]:
len(df[df['Tuberculosis']==1.0])

70

In [27]:
ate = df[df['Atelectasis']==1.0]

In [28]:
ate[ate['Atelectasis Balanced Score']>= Tr

SyntaxError: unexpected EOF while parsing (3798430437.py, line 1)

In [ ]:
df[df['Mass']==1.0]['Page in CVAT']

In [ ]:
CVATPage(FN(dfwoNM,'Atelectasis'))